# Modeling of Pope tensors with e3nn
We demonstrate that e3nn can learn operations that are equivalent to preprocessing proposed by Ling et al. in Reynolds averaged turbulence modelling using deep neural networks with embedded invariance - an approach still prevalent in turbulence closure modeling.

First, we randomly create tensors, which we will later interpret as S and R tensors.

In [1]:
import e3nn
import torch

In [2]:
# Create random variables that represent different cases of velocity grad
gradU = (torch.rand(90000).reshape(-1, 3, 3)*2)-1
# Compute S and R
S = (gradU + gradU.transpose(2, 1))/2 # S = (gradU + gradU.T)/2
R = (gradU - gradU.transpose(2, 1))/2 # R = (gradU - gradU.T)/2

For any pair of R and S, we can compute the tensor T9 from Pope et al. as follows.

In [3]:
# Get T(9) through symbolic computation
Ssq = torch.einsum('bij, bjk -> bik', S, S) # b is the batch dimension for the matrix multiplication ij, jk -> ik
Rsq = torch.einsum('bij, bjk -> bik', R, R)
RsqSsq = torch.einsum('bij, bjk -> bik', Rsq, Ssq)
SsqRsq = torch.einsum('bij, bjk -> bik', Ssq, Rsq)
trSsqRsq = torch.einsum('bij, bij -> b', SsqRsq, SsqRsq)
T9 = RsqSsq + RsqSsq - (torch.eye(3)[None, :, :] * trSsqRsq[:, None, None])

In [4]:
T9.shape

torch.Size([10000, 3, 3])

Notably, the computation of T9 demands all of the operations used for basis tensors: matrix multiplication of tensors, taking trace of a tensor and addition/subtraction.

We create a custom e3nn-based architecture with atomic operations that can reflect those operations. Note that every layer in this model has just enough memory to store different stages of computation as shown in fig. 3 of our paper.

In [5]:
# Define the minimal architecture to achieve the same result on cartesian tensors
from torch.nn import Module
from e3nn.io import CartesianTensor
from e3nn import o3

class T9Model(Module):
    def __init__(self):
        super(T9Model, self).__init__()
        # CartesianTensor objects are not trainable and will serve for transition between Cartesian and spherical space
        self.ct = CartesianTensor('ij') # a class to represent gradU as 1x0e + 1x1e + 1x2e Irrep tensors
        self.sct = CartesianTensor('ij=ji') # a class to represent a symmetric tensor as 1x0e + 1x2e
        self.act = CartesianTensor('ij=-ji') # a class to represent a skew-symmetric tensor as 1x1e
        # We could use two CartesianTensor interfaces to separately transform S and R.
        # Instead, we will directly decompose the full velocity gradient tensor gradU into irreps, which is equivalent
        # self.ctS = CartesianTensor('ij=ji')
        
        # We prepare the TensorProduct layers that, in the spherical space, will learn operations homeomorphic to Cartesian matrix multiplication
        # We also prepare a Linear layer to handle the summation
        self.tp1 = o3.FullyConnectedTensorProduct('0e + 2e', '0e + 2e', '0e + 1e + 2e') # the output irreps are determined by transformation properties of Ssq and Rsq under rotation
        self.tp1a = o3.FullyConnectedTensorProduct('1e', '1e', '0e + 1e + 2e')
        self.tp2 = o3.FullyConnectedTensorProduct('0e + 1e + 2e', '0e + 1e + 2e', '0e + 1e + 2e')
        self.tp2a = o3.FullyConnectedTensorProduct('0e + 1e + 2e', '0e + 1e + 2e', '0e + 1e + 2e')
        self.tp3 = o3.FullyConnectedTensorProduct('0e + 1e + 2e', '0e + 1e + 2e', '0e')
        self.lin = o3.Linear('0e + 1e + 2e + 0e + 1e + 2e + 0e', '0e + 1e + 2e')
        
        # The variables below will be filled with cached values to accelerate the forward call - this is just a technicality of e3nn.io.CartesianTensor
        self.rtp = None
        self.srtp = None
        self.artp = None

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def forward(self, x):
        if self.rtp is None:
            self.rtp = self.ct.reduced_tensor_products(torch.tensor([0., 1., 2.], device=x.device))
        if self.srtp is None:
            self.srtp = self.sct.reduced_tensor_products(torch.tensor([0., 1., 2.], device=x.device))
        if self.artp is None:
            self.artp = self.act.reduced_tensor_products(torch.tensor([0., 1., 2.], device=x.device))
         
        # Transformation from Cartesian space
        x_sym = self.sct.from_cartesian(x, rtp=self.srtp) # get S
        x_asym = self.act.from_cartesian(x, rtp=self.artp) # get R
        #x_spher = self.ct.from_cartesian(x, rtp=self.rtp) This is how we would decompose gradU in a single step
        x1 = self.tp1(x_sym, x_sym) # (Train to) compute S^2 (or equivalently descriptive variable)
        x1a = self.tp1a(x_asym, x_asym) # (Train to) comput R^2 (or equivalently descriptive variable)
        x2 = self.tp2(x1, x1a)
        x2a = self.tp2a(x1, x1a)
        x2tr = self.tp3(x2, x2a)
        out = self.lin(torch.cat([x2, x2a, x2tr], dim=1))
        #out = self.lin(x2)
        
        # Transformation back to Cartesian space
        out_cart = self.ct.to_cartesian(out, rtp=self.rtp)
        return out_cart

We demonstrate that the proposed simplistic architecture can perform the operation equivalent to computation of Pope's 9th tensor, even though the variables are encoded in the spherical space. The two cells below involve training with moderate learning rate, with the first one employing weight decay to sparsify the randomly-initialized parameters of e3nn operations.

The architecture we defined contains 49 trainable parameters, yet fits the 10 000 datapoints easily. Ultimately, it shall be validated on the test set of 100 000 datapoints. We argue that the number of examples does not matter, as the network can model the operations used in Pope's preprocessing.

Conversely, e3nn-based architectures, as proposed in the paper, can learn operations equivalent to those employed by tensor basis approach, making our proposed model a superset of TBNN. One has to note, however, we did not emphasize exactly matching to Pope's tensor basis: in our main model, e3nn operations are intertwined with nonlinear activation functions. Our approach is more akin to typical design of neural networks: nonlinear activation functions, batch normalization and inference distributed among different neurons make for a more amicable ground for training than symbolic operations devised by Pope.

In [6]:
from tqdm import tqdm
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
model = T9Model()
model.train()
loader = DataLoader(TensorDataset(gradU, T9), batch_size=16)
optimizer = Adam(model.parameters(), lr=10e-4, weight_decay=1e-7)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.97)
epochs = 20
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(f'Number of model params: {sum(p.numel() for p in model.parameters())}')
for i in range(epochs):
    error = 0
    steps = 0
    for batch in tqdm(loader, total=len(loader)):
        x, y = batch
        optimizer.zero_grad()
        out = model(x)
        loss = F.mse_loss(out, y)
        loss.backward()
        optimizer.step()
        error += loss.detach()
        steps += 1
    print(f'Epoch {i}; total error: {error}, average: {error/steps}')
    scheduler.step()

c:\users\grzegorzkaszuba\appdata\local\programs\python\python38\lib\site-packages\torch\jit\_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Number of model params: 49


100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:25<00:00, 24.05it/s]


Epoch 0; total error: 217.21981811523438, average: 0.34755170345306396


100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:24<00:00, 26.01it/s]


...

Epoch 17; total error: 2.7500176429748535, average: 0.004400028381496668


100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:08<00:00, 69.78it/s]


Epoch 18; total error: 2.330662965774536, average: 0.003729060757905245


100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:08<00:00, 70.55it/s]

Epoch 19; total error: 1.8846261501312256, average: 0.0030154017731547356


In [7]:
optimizer = Adam(model.parameters(), lr=1e-4)
epochs = 100
for i in range(epochs):
    error = 0
    steps = 0
    for batch in tqdm(loader, total=len(loader)):
        x, y = batch
        optimizer.zero_grad()
        out = model(x)
        loss = F.mse_loss(out, y)
        loss.backward()
        optimizer.step()
        error += loss.detach()
        steps += 1
    print(f'Epoch {i}; total error: {error}, average: {error/steps}')

100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:08<00:00, 71.21it/s]


Epoch 0; total error: 1.5647218227386475, average: 0.002503554802387953


100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:08<00:00, 69.54it/s]


Epoch 1; total error: 1.4653446674346924, average: 0.002344551496207714


100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:08<00:00, 69.89it/s]


...

Epoch 97; total error: 8.7444295786554e-06, average: 1.3991087399745084e-08


100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:08<00:00, 70.69it/s]


Epoch 98; total error: 1.3035785741521977e-05, average: 2.0857257254647266e-08


100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:08<00:00, 70.04it/s]

Epoch 99; total error: 5.050282652518945e-06, average: 8.080451863179405e-09


We verify that this minimalist architecture is just applicable to larger (100 thousand) portion of never-seen randomly generated examples.

In [9]:
# Create a bigger test set
gradU = (torch.rand(900000).reshape(-1, 3, 3)*2)-1
#gradU = (torch.rand(9000000).reshape(-1, 3, 3))
# Compute S and R
S = (gradU + gradU.transpose(2, 1))/2 # S = (gradU + gradU.T)/2
R = (gradU - gradU.transpose(2, 1))/2 # R = (gradU - gradU.T)/2

# Get T(9) through symbolic computation
Ssq = torch.einsum('bij, bjk -> bik', S, S) # b is the batch dimension for the matrix multiplication ij, jk -> ik
Rsq = torch.einsum('bij, bjk -> bik', R, R)
RsqSsq = torch.einsum('bij, bjk -> bik', Rsq, Ssq)
SsqRsq = torch.einsum('bij, bjk -> bik', Ssq, Rsq)
trSsqRsq = torch.einsum('bij, bij -> b', SsqRsq, SsqRsq)
T9 = RsqSsq + RsqSsq - (torch.eye(3)[None, :, :] * trSsqRsq[:, None, None])

test_loader = DataLoader(TensorDataset(gradU, T9), batch_size=64)

model.eval()
error = 0
steps = 0
trues = []
preds = []
for batch in tqdm(test_loader, total=len(test_loader)):
    x, y = batch
    out = model(x)
    loss = F.mse_loss(out, y)
    error += loss.detach()
    steps += 1
    # let's also express the testing error in terms of r^2 - for that 
    trues.append(y.reshape(-1))
    preds.append(out.detach().reshape(-1))
trues = torch.cat(trues)
preds = torch.cat(preds)
print(f'Test epoch; total error: {error}, average: {error/steps}')

100%|█████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:09<00:00, 165.52it/s]

Test epoch; total error: 5.057370966454755e-09, average: 3.23568190470358e-12


In [13]:
def r2_score(y_true, y_pred):
    ss_res = torch.sum((y_true - y_pred) ** 2)
    y_true_mean = torch.mean(y_true)
    ss_tot = torch.sum((y_true - y_true_mean) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2.item()
print(np.array(r2_score(trues, preds)))

1.0


In [ ]:
sum(p.numel() for p in model.parameters())

In [ ]:
def e3_operations(x_sym, x_asym):
    x1 = model.tp1(x_sym, x_sym)
    x1a = model.tp1a(x_asym, x_asym)
    x2 = model.tp2(x1, x1a)
    x2a = model.tp2a(x1, x1a)
    x2tr = model.tp3(x2, x2a)
    out = model.lin(torch.cat([x2, x2a, x2tr], dim=1))
    return out

from e3nn.util.test import equivariance_error
model.eval()
for batch in test_loader:
    x, y = batch
    err = equivariance_error(
        e3_operations,
        args_in=[model.sct.from_cartesian(x, rtp=model.srtp), model.act.from_cartesian(x, rtp=model.artp)],
        irreps_in=['0e + 2e', '1e'],
        irreps_out=['0e + 1e + 2e']
    )
    break
print(err)